In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import  cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from skimage.feature import hog, local_binary_pattern

In [11]:
genuine_path = r'archive(1)\signatures\full_org'
forged_path = r'archive(1)\signatures\full_forg'

In [21]:
def extract_features(img):
    img=cv2.resize(img,(256,128))
    hog_features = hog(img, pixels_per_cell=(16, 16), cells_per_block=(2, 2), feature_vector=True)
    lbp = local_binary_pattern(img, P=24, R=3, method="uniform")
    (lbp_hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= lbp_hist.sum()
    moments = cv2.moments(img)
    hu_moments = cv2.HuMoments(moments).flatten()
    features = np.hstack([hog_features, lbp_hist, hu_moments])
    return features


In [22]:
def load_images(folder,label):
    data=[]
    labels=[]
    for filename in os.listdir(folder):
        img_path=os.path.join(folder,filename)
        img=cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
        if img is not None:
            features=extract_features(img)
            data.append(features)
            labels.append(label)
    return np.array(data),np.array(labels)


In [23]:
data_gen,label_gen=load_images(genuine_path,label=0)
data_forged,label_forged=load_images(forged_path,label=1)

In [24]:
X = np.concatenate((data_gen, data_forged), axis=0)
y = np.concatenate((label_gen, label_forged), axis=0)

In [25]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=67)

In [27]:
X_train.shape

(2112, 3813)

In [28]:
svm = SVC(kernel='linear', C=1.0)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

In [29]:
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))

SVM Accuracy: 0.9829545454545454


In [34]:
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [35]:
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

Random Forest Accuracy: 0.990530303030303


In [32]:
print("\nSVM Classification Report:")
print(classification_report(y_test, y_pred_svm))


SVM Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       259
           1       0.99      0.98      0.98       269

    accuracy                           0.98       528
   macro avg       0.98      0.98      0.98       528
weighted avg       0.98      0.98      0.98       528



In [33]:
print("\nRandom Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))


Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       259
           1       1.00      0.98      0.99       269

    accuracy                           0.99       528
   macro avg       0.99      0.99      0.99       528
weighted avg       0.99      0.99      0.99       528

